In [1]:
#Total income for each month of the year and Average monthly income per year
#▪Input: a (structured) textual csv files containing the daily income of a company
#▪Each line of the files has the following format: 
#    date\tdaily income\n
#Output:
#▪Total income for each month of the year
#▪Average monthly income for each year

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Study\\anaconda3\\lib\\site-packages\\pyspark'

In [3]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\Study\Documents\Jupyter Notebooks\PoliTO\1-30 MapReduce Exercises\Exercise-8


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Exercise 8") \
    .getOrCreate()
sc = spark.sparkContext

In [5]:
spark

In [6]:
sc

<SparkContext master=local[*] appName=Exercise >

In [38]:
rdd8_raw = sc.textFile('file_input_8.txt')

In [39]:
rdd8_raw.take(10)

['2015-11-01\t1000',
 '2015-11-02\t1305',
 '2015-12-01\t500',
 '2015-12-02\t750',
 '2016-01-01\t345',
 '2016-01-02\t1145',
 '2016-02-03\t200',
 '2016-02-04\t500']

In [40]:
rdd8_processing = rdd8_raw.map(lambda row: row.split('\t'))
rdd8_processing.take(10)

[['2015-11-01', '1000'],
 ['2015-11-02', '1305'],
 ['2015-12-01', '500'],
 ['2015-12-02', '750'],
 ['2016-01-01', '345'],
 ['2016-01-02', '1145'],
 ['2016-02-03', '200'],
 ['2016-02-04', '500']]

In [10]:
rdd8_processing = rdd8_processing.map(lambda row: (row[0][0:7], float(row[1]) ) )

In [41]:
#rdd8_raw.first()[0:7]
rdd8_processing.take(10)

[['2015-11-01', '1000'],
 ['2015-11-02', '1305'],
 ['2015-12-01', '500'],
 ['2015-12-02', '750'],
 ['2016-01-01', '345'],
 ['2016-01-02', '1145'],
 ['2016-02-03', '200'],
 ['2016-02-04', '500']]

In [42]:
rdd8_result_sum = rdd8_processing.reduceByKey(lambda a,b: (a+b) )

In [43]:
rdd8_result_sum.take(10)

[('2015-12-01', '500'),
 ('2015-12-02', '750'),
 ('2016-01-01', '345'),
 ('2016-01-02', '1145'),
 ('2016-02-04', '500'),
 ('2015-11-01', '1000'),
 ('2015-11-02', '1305'),
 ('2016-02-03', '200')]

In [23]:
rdd8_processing_year = rdd8_processing.map(lambda row : (row[0][0:4],row[1], 1 ))  #: (row[0][0:7], float(row[1])
rdd8_processing_year.take(10)

#.countByKey()

[('2015', 1000.0, 1),
 ('2015', 1305.0, 1),
 ('2015', 500.0, 1),
 ('2015', 750.0, 1),
 ('2016', 345.0, 1),
 ('2016', 1145.0, 1),
 ('2016', 200.0, 1),
 ('2016', 500.0, 1)]

In [49]:
rdd8_processing_year_v2 = rdd8_processing.map(lambda row : (row[0][0:7],1 ))

rdd8_processing_year_v2.take(10)



[('2015-11', 1),
 ('2015-11', 1),
 ('2015-12', 1),
 ('2015-12', 1),
 ('2016-01', 1),
 ('2016-01', 1),
 ('2016-02', 1),
 ('2016-02', 1)]

In [34]:
rdd8_year_sum_amount = rdd8_processing_year.map(lambda row : (row[0],row[1])).reduceByKey(lambda a,b : a+b)
dict_rdd8_year_sum_amount = rdd8_year_sum_amount.collectAsMap()

In [54]:
rdd8_year_sum_count = rdd8_processing_year_v2.reduceByKey(lambda a,b : 1)
rdd8_year_sum_count = rdd8_year_sum_count.map(lambda row : (row[0][0:4],row[1] )).reduceByKey(lambda a,b : a+b)
rdd8_year_sum_count.take(10)
#.map(lambda row : (row[0],row[2])).reduceByKey(lambda a,b : a+b)
dict_rdd8_year_sum_count = rdd8_year_sum_count.collectAsMap()
dict_rdd8_year_sum_count

{'2015': 2, '2016': 2}

In [55]:
print(dict_rdd8_year_sum_amount)
print(dict_rdd8_year_sum_count)
dict_result = {}
for key in dict_rdd8_year_sum_amount.keys():
    dict_result[key] =  dict_rdd8_year_sum_amount[key] / dict_rdd8_year_sum_count[key]
print (dict_result)

{'2015': 3555.0, '2016': 2190.0}
{'2015': 2, '2016': 2}
{'2015': 1777.5, '2016': 1095.0}
